DOWNLOADS

In [ ]:
!pip install accelerate peft bitsandbytes transformers

In [ ]:
!pip install -U datasets

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA: True
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
import numpy as np
import pandas as pd

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

NumPy version: 2.0.2
Pandas version: 2.2.2


In [ ]:
!pip install flash-attn --no-build-isolation

In [ ]:
!pip install trl

IMPORT AND CONFIGS

In [ ]:
# ft_ino_llama31_qlora_trainer.py
import os, re, math, random
import torch
from datasets import load_dataset
from typing import List, Dict
from tqdm import tqdm

from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

SEED = 42
random.seed(SEED); torch.manual_seed(SEED)

In [ ]:
#MODEL_NAME = os.getenv("LLM_MODEL", "meta-llama/Llama-3.2-3B-Instruct")  # veya "Meta-Llama-3.2-3B-Instruct"
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
BATCH_SIZE = 4
MAX_NEW_TOKENS = 24
TEMPERATURE = 0.0
TOP_P = 1.0
SEED = 42

DATASET

In [ ]:
# Dataset
data_files = {
    "train": "train.jsonl",
    "test": "test.jsonl"
}
from datasets import load_dataset

dataset = load_dataset("bengisucam/LLL_INO-tagged", data_files=data_files, cache_dir=None, download_mode='force_redownload')

# Access the splits directly from the loaded dataset
train_dataset = dataset["train"]
val_dataset = dataset["test"]

print(train_dataset[:2])

train.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/139 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25 [00:00<?, ? examples/s]

{'Unnamed: 0': [0, 1], 'Gene1': ['KinC', 'KinD'], 'Gene2': ['Spo0A~P', 'Spo0A~P'], 'Keywords': ['responsible, production', 'responsible, production'], 'Direct dependency relations': ['nmod(responsible, production)', 'nmod(responsible, production)'], 'INO Type': ['regulation of translation', 'regulation of translation'], 'INO ID': ['INO_0000034', 'INO_0000034'], 'Num. Keywords': ['2', '2'], 'Sentence': ['In vivo studies of the activity of four of the kinases, KinA, [PROTEIN1] KinC [/PROTEIN1], KinD (ykvD) and KinE (ykrQ), using abrB transcription as an indicator of [PROTEIN2] Spo0A~P [/PROTEIN2] level, revealed that KinC and KinD were responsible for Spo0A~P production during the exponential phase of growth in the absence of KinA and KinB.', 'In vivo studies of the activity of four of the kinases, KinA, KinC, [PROTEIN1] KinD [/PROTEIN1] (ykvD) and KinE (ykrQ), using abrB transcription as an indicator of [PROTEIN2] Spo0A~P [/PROTEIN2] level, revealed that KinC and KinD were responsible f

HELPER FUNCTIONS

In [ ]:
def normalize_keyword_set(value):
    if value is None or (isinstance(value, float) and math.isnan(value)):
        return set()
    if isinstance(value, list):
        toks = [str(x).strip().lower() for x in value if str(x).strip()]
    else:
        toks = [t.strip().lower() for t in str(value).split(",") if t.strip()]
    toks = [t for t in toks if t not in {"none","no","n/a"}]
    return set(toks)

def keywords_to_str(value):
    return ", ".join(sorted(normalize_keyword_set(value))) if value is not None else ""

def parse_model_output_to_set(text):
    if text is None:
        return set()
    line = str(text).strip().splitlines()[0]
    toks = [t.strip().lower() for t in line.split(",") if t.strip()]
    cleaned = []
    for t in toks:
        t = re.sub(r"^(answer|keywords?|key word\(s\)|output)\s*[:\-]\s*", "", t).strip()
        if t:
            cleaned.append(t)
    return set(cleaned)

def micro_prf1(golds, preds):
    tp = fp = fn = 0
    for g, p in zip(golds, preds):
        tp += len(g & p)
        fp += len(p - g)
        fn += len(g - p)
    prec = tp/(tp+fp) if (tp+fp) else 0.0
    rec  = tp/(tp+fn) if (tp+fn) else 0.0
    f1   = 2*prec*rec/(prec+rec) if (prec+rec) else 0.0
    return {"precision": prec, "recall": rec, "f1": f1, "tp":tp,"fp":fp,"fn":fn}

MODEL CONFIGS

In [ ]:
#MODEL_NAME = os.getenv("LLM_MODEL", "meta-llama/Llama-3.2-3B-Instruct")
USE_4BIT = True  # GPU yoksa False yap
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

def get_bnb():
    if not USE_4BIT: return None
    bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    return BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if bf16_ok else torch.float16,
    )

PROMPT

In [ ]:
SYSTEM_MSG = (
    "You are a biomedical IE assistant. Return ONLY the interaction keyword(s) "
    "as a comma-separated list. Do not add extra words."
)
INSTRUCTION = (
    "Identify the interaction keyword(s) that describe the relation between the two proteins "
    "tagged with [PROTEIN1]...[/PROTEIN1] and [PROTEIN2]...[/PROTEIN2] in the sentence. "
    "Return only a comma-separated list (e.g., 'transcription, depends'). If none, output 'none'."
)

def to_messages(sentence:str, gold_csv:str):
    return [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": f"{INSTRUCTION}\n\nSentence: {sentence}"},
        {"role": "assistant", "content": gold_csv},
    ]

TOKENIZERS

In [ ]:
def build_tokenizer_train():
    tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "right"  # training
    return tok

def build_tokenizer_infer():
    tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"   # generation
    return tok

QLORA CONFIG

In [ ]:
# Llama katman adları: q_proj,k_proj,v_proj,o_proj,up_proj,down_proj,gate_proj
TARGET_MODULE_KEYWORDS = ["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"]

def create_peft_config():
    return LoraConfig(
        r=8, lora_alpha=32, lora_dropout=0.05,
        task_type="CAUSAL_LM", bias="none",
        target_modules=TARGET_MODULE_KEYWORDS,
    )

PREPROCESSING

In [ ]:
def render_with_template(ds, tokenizer):
    def _fmt(ex):
        messages = to_messages(ex)
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False  # gold assistant var → loss sadece burada
        )
        return {"text": text}
    return ds.map(_fmt, desc="render chat template")

MODEL PREPERATION

In [ ]:
def load_base_model():
    bnb = get_bnb_config()
    attn_kwargs = {}
    if torch.cuda.is_available():
        attn_kwargs["attn_implementation"] = "flash_attention_2"
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=True,
        **attn_kwargs
    )
    model.config.use_cache = False  # gradient checkpointing ile uyum
    return model

def attach_lora(model):
    lora = LoraConfig(
        r=8, lora_alpha=32, lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj"
        ],
    )
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora)
    model.print_trainable_parameters()
    return model

COLLECTOR

In [ ]:
class CompletionOnlyCollator:
    """
    - Chat template ile tüm diyaloğu tek stringe çevirir.
    - Labels: sadece assistant cevabı (gold) tokenlarında loss; diğerleri -100.
    """
    def __init__(self, tokenizer, max_len=1024):
        self.tok = tokenizer
        self.max_len = max_len

    def __call__(self, batch: List[Dict]):
        texts = []
        prefix_token_lens = []

        for ex in batch:
            gold_csv = keywords_to_str(ex["Keywords"])
            msgs = to_messages(ex["Sentence"], gold_csv)
            text_full = self.tok.apply_chat_template(
                msgs, tokenize=False, add_generation_prompt=False
            )
            texts.append(text_full)

            # Assistant içeriğini boş verip prefix uzunluğunu bul
            msgs_no_answer = to_messages(ex["Sentence"], gold_csv="")
            prefix = self.tok.apply_chat_template(
                msgs_no_answer, tokenize=False, add_generation_prompt=False
            )
            prefix_ids = self.tok(prefix, add_special_tokens=False).input_ids
            prefix_token_lens.append(len(prefix_ids))

        enc = self.tok(texts, padding=True, truncation=True,
                       max_length=self.max_len, return_tensors="pt")

        labels = enc["input_ids"].clone()
        # Right padding kullandığımız için prefix uzunluğu, sequence başından itibaren mask sınırı
        for i, start_idx in enumerate(prefix_token_lens):
            labels[i, :start_idx] = -100  # system+user kısmı maskeli

        enc["labels"] = labels
        return enc

TRAINING

In [ ]:
def train(output_dir="./ft_ino_llama31_lora"):
    bnb = get_bnb()
    attn_kwargs = {}
    if torch.cuda.is_available():
        attn_kwargs["attn_implementation"] = "flash_attention_2"

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, device_map="auto",
        quantization_config=bnb, trust_remote_code=True, **attn_kwargs
    )
    model.config.use_cache = False
    model.gradient_checkpointing_enable()

    # QLoRA hazırlığı + adapter ekleme
    model = prepare_model_for_kbit_training(model)
    peft_cfg = create_peft_config()
    model = get_peft_model(model, peft_cfg)
    model.print_trainable_parameters()

    tok_tr = build_tokenizer_train()
    collator = CompletionOnlyCollator(tok_tr, max_len=1024)

    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=1e-4,         # küçük veri -> 1e-4 daha stabil
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        logging_steps=10,
        eval_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to="none",
        bf16=torch.cuda.is_available() and torch.cuda.is_bf16_supported(),
        fp16=not (torch.cuda.is_available() and torch.cuda.is_bf16_supported()),
        seed=SEED,
        remove_unused_columns=False
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collator,
        tokenizer=tok_tr,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )

    trainer.train()
    trainer.save_model(os.path.join(output_dir, "final"))
    tok_tr.save_pretrained(os.path.join(output_dir, "final"))
    return os.path.join(output_dir, "final")

EVAL

In [ ]:
MAX_NEW_TOKENS = 24
BATCH_SIZE = 8

def build_eval_chat(sentence):
    return [
        {"role":"system","content":SYSTEM_MSG},
        {"role":"user","content":f"{INSTRUCTION}\n\nSentence: {sentence}"},
    ]

@torch.no_grad()
def generate_batch(enc, model, tok):
    return model.generate(
        **enc,
        do_sample=False,              # uyarı olmasın
        max_new_tokens=MAX_NEW_TOKENS,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
    )

@torch.no_grad()
def evaluate_split(split_dataset, model_path_or_name):
    tok = build_tokenizer_infer()
    bnb = get_bnb()
    attn_kwargs = {}
    if torch.cuda.is_available():
        attn_kwargs["attn_implementation"] = "flash_attention_2"

    model = AutoModelForCausalLM.from_pretrained(
        model_path_or_name, device_map="auto",
        quantization_config=bnb, trust_remote_code=True, **attn_kwargs
    )
    model.eval()

    inputs, gold_sets = [], []
    for sent, gold in zip(split_dataset["Sentence"], split_dataset["Keywords"]):
        messages = build_eval_chat(sent)
        prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs.append(prompt)
        gold_sets.append(normalize_keyword_set(gold))

    preds, raws = [], []
    for i in tqdm(range(0, len(inputs), BATCH_SIZE), desc="Evaluating"):
        batch_texts = inputs[i:i+BATCH_SIZE]
        enc = tok(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)
        out = generate_batch(enc, model, tok)
        for j in range(out.size(0)):
            gen_ids = out[j, enc["input_ids"].size(1):]
            text = tok.decode(gen_ids, skip_special_tokens=True).strip()
            raws.append(text)
            preds.append(parse_model_output_to_set(text))

    scores = micro_prf1(gold_sets, preds)
    return scores, raws, gold_sets, preds

MAIN RUN

In [ ]:
out_dir = train()
scores, raw, gold, pred = evaluate_split(val_dataset, out_dir)
print("\n=== FT (QLoRA + assistant-only loss) KEYWORD-LEVEL ===")
print({k: round(v,4) if isinstance(v,float) else v for k,v in scores.items()})
for i in range(min(5, len(val_dataset))):
    print(f"\nSentence: {val_dataset[i]['Sentence']}")
    print(f"Gold: {gold[i]}")
    print(f"Pred: {pred[i]} | Raw='{raw[i]}'")

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

trainable params: 6,522,880 || all params: 1,006,408,832 || trainable%: 0.6481


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

/tmp/ipython-input-3710333686.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `flash_attention_2`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
10,7.578600,1.561222
20,0.682700,0.334145
30,0.214500,0.213021
40,0.113200,0.200076


Evaluating: 100%|██████████| 4/4 [00:16<00:00,  4.12s/it]


=== FT (QLoRA + assistant-only loss) KEYWORD-LEVEL ===
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'tp': 0, 'fp': 26, 'fn': 46}

Sentence: In addition to the typical [PROTEIN1] sigmaB [/PROTEIN1]-dependent, stress- and starvation-inducible pattern, [PROTEIN2] yvyD [/PROTEIN2] is also induced in response to amino acid depletion.
Gold: {'inducible', 'dependent'}
Pred: {'none'} | Raw='none'

Sentence: We overproduced and purified sigma(X) from Escherichia coli and demonstrate that in vitro, both sigma(A) and [PROTEIN1] sigma(X) [/PROTEIN1] holoenzymes recognize promoter elements within the sigX-[PROTEIN2] ypuN [/PROTEIN2] control region.
Gold: {'promoter', 'recognize'}
Pred: {'none'} | Raw='none'

Sentence: The sigmaB-dependent promoter drives expression of yvyD under stress conditions and after glucose starvation, whereas a [PROTEIN1] sigmaH [/PROTEIN1]-dependent promoter is responsible for [PROTEIN2] yvyD [/PROTEIN2] transcription following amino acid limitation.
Gold: {'expression', 